In [1]:
using DifferentialEquations, DiffEqProblemLibrary, ParameterizedFunctions, Plots
gr()

van = @ode_def_noinvhes VanDerPol2 begin
  dy = μ*((1-x^2)*y - x)
  dx = 1*y
end μ=>1.

prob = ODEProblem(VanDerPol2(μ=1e6),[0;2.],(0.0,6.3))
abstols = 1./10.^(6:9)
reltols = 1./10.^(3:6)

sol = solve(prob,CVODE_BDF,abstol=1/10^14,reltol=1/10^14)
test_sol = TestSolution(sol.t,sol.u)

DiffEqDevTools.TestSolution

## Graphs 

These graphs are to show that the methods compute the correct solution.

In [2]:
sol = solve(prob,Rosenbrock23,abstol=abstols[3],reltol=reltols[3],maxiters=Int(1e5))
plot(sol,denseplot=false,ylim=[-4;4])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 21.7148,194.444 21.7148,194.445 21.7148,194.45 21.7148,194.494 21.7148,194.532 21.7148,194.562 21.7148,194.708 21.7148,194.784 21.7148,195.057 21.7148,195.209 
 21.7148,195.608 21.7148,195.863 21.7148,196.375 21.7148,196.745 21.7148,197.353 21.7148,197.836 21.7148,198.522 21.7148,199.109 21.7148,199.855 21.7148,200.528 
 21.7148,201.319 21.7148,202.058 21.7148,202.881 21.7148,203.666 21.7148,204.507 21.7148,205.32 21.7148,206.168 21.7148,206.994 21.7148,207.838 21.7148,208.662 
 21.7148,209.492 21.7148,210.303 21.7148,211.111 21.7148,211.9 21.7148,212.679 21.7148,213.437 21.7148,214.179 21.7148,214.9 21.7148,215.603 21.7148,216.282 
 21.7148,216.94 21.7148,217.573 21.7148,218.183 21.7148,218.769 21.7148,219.33 21.7148,219.865 21.7148,220.376 21.7148,220.861 21.7148,221.321 21.7148,221.756 
 21.7148,222.166 21.7149,222.551 21.7149,222.913 21.7149,223.25 21.7149,223.565 21.7149,223.856 21.7149,224.126 21.7149,224.375 21.7149,224.602 21.7149,224.81 
 21.7149,224.999 21.7149,225.17 21.7149,225.324 21.7149,225.461 21.7149,225.582 21.7149,225.689 21.7149,225.783 21.7149,225.864 21.7149,225.933 21.7149,225.991 
 21.7149,226.04 21.715,226.08 21.715,226.112 21.715,226.137 21.715,226.157 21.715,226.171 21.715,226.181 21.715,226.188 21.7151,226.192 21.7151,226.194 
 21.7151,226.195 21.7152,226.196 21.7152,226.196 21.7154,226.196 21.7163,226.196 21.7192,226.197 21.7345,226.199 21.7642,226.205 21.8166,226.215 21.8973,226.231 
 22.0133,226.254 22.167,226.284 22.3613,226.322 22.5947,226.367 22.8665,226.421 23.1724,226.482 23.5099,226.549 23.8739,226.623 24.2615,226.701 24.668,226.784 
 25.0909,226.871 25.5262,226.962 25.972,227.056 26.4252,227.152 26.8848,227.25 27.3481,227.35 27.8146,227.452 28.2825,227.555 28.7514,227.66 29.22,227.765 
 29.6882,227.872 30.1549,227.979 30.6204,228.087 31.0836,228.196 31.545,228.305 32.0036,228.415 32.46,228.526 32.9134,228.637 33.3643,228.748 33.8119,228.86 
 34.257,228.972 34.6987,229.085 35.1377,229.198 35.5734,229.312 36.0063,229.426 36.4359,229.54 36.8627,229.655 37.2862,229.771 37.7069,229.886 38.1243,230.002 
 38.5389,230.119 38.9502,230.235 39.3588,230.353 39.7642,230.47 40.1669,230.588 40.5664,230.706 40.9631,230.825 41.3568,230.944 41.7478,231.064 42.1357,231.184 
 42.521,231.304 42.9032,231.425 43.2829,231.546 43.6596,231.667 44.0337,231.789 44.4048,231.911 44.7734,232.034 45.1392,232.157 45.5024,232.281 45.8628,232.404 
 46.2207,232.529 46.5759,232.653 46.9285,232.778 47.2785,232.904 47.626,233.03 47.9708,233.156 48.3133,233.283 48.6531,233.41 48.9905,233.538 49.3253,233.666 
 49.6578,233.794 49.9878,233.923 50.3154,234.052 50.6405,234.182 50.9634,234.312 51.2837,234.442 51.6019,234.573 51.9176,234.704 52.2311,234.836 52.5422,234.968 
 52.8511,235.101 53.1576,235.234 53.462,235.368 53.7641,235.502 54.064,235.636 54.3617,235.771 54.6573,235.906 54.9506,236.042 55.2419,236.178 55.5309,236.315 
 55.8179,236.452 56.1028,236.589 56.3856,236.727 56.6663,236.865 56.945,237.004 57.2216,237.144 57.4962,237.283 57.7687,237.424 58.0394,237.564 58.308,237.706 
 58.5747,237.847 58.8393,237.989 59.1021,238.132 59.3629,238.275 59.6219,238.418 59.8789,238.562 60.1341,238.707 60.3874,238.852 60.6389,238.997 60.8885,239.143 
 61.1363,239.289 61.3823,239.436 61.6265,239.583 61.8688,239.731 62.1095,239.879 62.3484,240.028 62.5855,240.178 62.8209,240.327 63.0546,240.478 63.2866,240.628 
 63.5169,240.78 63.7454,240.931 63.9724,241.084 64.1977,241.236 64.4213,241.39 64.6433,241.543 64.8637,241.698 65.0825,241.852 65.2997,242.008 65.5153,242.163 
 65.7293,242.32 65.9418,242.477 66.1527,242.634 66.3621,242.792 66.5699,242.95 66.7763,243.109 66.9811,243.269 67.1844,243.429 67.3863,243.589 67.5867,243.75 
 67.7857,243.912 67

In [ ]:
sol = solve(prob,Rosenbrock32)
plot(sol,denseplot=false,ylim=[-4;4])

Rosenbrock32 does not do well on oscillatory problems since it's not L-stable!

In [7]:
sol = solve(prob,ModifiedRosenbrockIntegrator,abstol=abstols[3],reltol=reltols[3],maxiters=Int(1e5))
plot(sol,ylim=[-4;4],denseplot=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 21.7148,196.949 21.7148,199.04 21.7148,201.029 21.7148,202.917 21.7148,204.708 21.7148,206.404 21.7148,208.008 21.7148,209.522 21.7148,210.949 21.7148,212.291 
 21.7148,213.551 21.7148,214.732 21.7148,215.836 21.7148,216.867 21.7148,217.826 21.7148,218.716 21.7148,219.54 21.7148,220.301 21.7148,221 21.7148,221.642 
 21.7148,222.227 21.7149,222.759 21.7149,223.241 21.7149,223.675 21.7149,224.064 21.7149,224.41 21.7149,224.715 21.7149,224.983 21.7149,225.216 21.7149,225.417 
 21.7149,225.587 21.7149,225.73 21.7149,225.848 21.7149,225.944 21.7149,226.02 21.715,226.078 21.715,226.122 21.715,226.153 21.715,226.173 21.715,226.185 
 21.7151,226.192 21.7151,226.195 21.7152,226.196 21.7153,226.196 21.7157,226.196 21.7177,226.196 21.7318,226.199 21.8931,226.23 22.3051,226.311 22.8741,226.423 
 23.5287,226.553 24.2276,226.694 24.9461,226.841 25.671,226.992 26.3945,227.145 27.1126,227.299 27.8232,227.454 28.5251,227.609 29.2178,227.765 29.9011,227.92 
 30.5749,228.076 31.2393,228.232 31.8944,228.389 32.5404,228.545 33.1773,228.702 33.8054,228.858 34.4247,229.015 35.0356,229.172 35.638,229.329 36.2321,229.486 
 36.8182,229.643 37.3962,229.801 37.9665,229.958 38.529,230.116 39.084,230.274 39.6316,230.432 40.1718,230.59 40.7049,230.748 41.2309,230.906 41.75,231.064 
 42.2623,231.223 42.7679,231.382 43.2669,231.541 43.7594,231.7 44.2455,231.859 44.7254,232.018 45.1991,232.177 45.6668,232.337 46.1285,232.497 46.5843,232.656 
 47.0344,232.816 47.4788,232.976 47.9176,233.137 48.3509,233.297 48.7788,233.457 49.2015,233.618 49.6188,233.779 50.0311,233.94 50.4382,234.101 50.8404,234.262 
 51.2377,234.423 51.6301,234.585 52.0178,234.746 52.4008,234.908 52.7792,235.07 53.1531,235.232 53.5225,235.394 53.8875,235.557 54.2481,235.719 54.6045,235.882 
 54.9567,236.045 55.3048,236.208 55.6488,236.371 55.9887,236.534 56.3247,236.697 56.6568,236.861 56.9851,237.024 57.3096,237.188 57.6304,237.352 57.9475,237.516 
 58.2609,237.681 58.5708,237.845 58.8772,238.01 59.1802,238.174 59.4797,238.339 59.7759,238.504 60.0687,238.67 60.3583,238.835 60.6447,239 60.9279,239.166 
 61.2079,239.332 61.4849,239.498 61.7588,239.664 62.0298,239.83 62.2978,239.997 62.5628,240.163 62.825,240.33 63.0844,240.497 63.3409,240.664 63.5947,240.831 
 63.8458,240.998 64.0942,241.166 64.34,241.334 64.5831,241.501 64.8237,241.669 65.0617,241.838 65.2972,242.006 65.5303,242.174 65.7609,242.343 65.9891,242.512 
 66.2149,242.681 66.4384,242.85 66.6596,243.019 66.8785,243.188 67.0951,243.358 67.3096,243.528 67.5218,243.698 67.7319,243.868 67.9398,244.038 68.1456,244.208 
 68.3494,244.379 68.551,244.55 68.7507,244.721 68.9484,244.892 69.144,245.063 69.3378,245.234 69.5296,245.406 69.7195,245.578 69.9075,245.749 70.0937,245.922 
 70.278,246.094 70.4606,246.266 70.6413,246.439 70.8203,246.611 70.9975,246.784 71.173,246.957 71.3469,247.13 71.519,247.304 71.6895,247.477 71.8584,247.651 
 72.0256,247.825 72.1912,247.999 72.3553,248.173 72.5178,248.348 72.6788,248.522 72.8382,248.697 72.9961,248.872 73.1526,249.047 73.3076,249.222 73.4611,249.398 
 73.6132,249.573 73.7639,249.749 73.9132,249.925 74.0611,250.101 74.2076,250.277 74.3528,250.454 74.4967,250.63 74.6392,250.807 74.7804,250.984 74.9204,251.161 
 75.0591,251.338 75.1965,251.516 75.3326,251.693 75.4676,251.871 75.6013,252.049 75.7338,252.227 75.8652,252.406 75.9953,252.584 76.1243,252.763 76.2522,252.942 
 76.3789,253.121 76.5045,253.3 76.629,253.479 76.7523,253.659 76.8746,253.839 76.9959,254.019 77.116,254.199 77.2352,254.379 77.3532,254.559 77.4703,254.74 
 77.5863,254.921 77.7014,255.102 77.8154,255.283 77.9285,255.464 78.0405,255.646 78.1517,255.827 78.2618,256.009 78.3711,256.191 78.4794,256.373 78.5868,256.556 
 78.6932,256.738 78.7988,256.921 78

In [8]:
sol = solve(prob,CVODE_BDF,abstol=abstols[3],reltol=reltols[3],maxiters=Int(1e5))
plot(sol,denseplot=false,ylim=[-4;4])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 21.7148,194.444 21.7148,194.444 21.7148,194.447 21.7148,194.456 21.7148,194.464 21.7148,194.473 21.7148,194.486 21.7148,194.51 21.7148,194.555 21.7148,194.65 
 21.7148,194.827 21.7148,195.003 21.7148,195.178 21.7148,195.352 21.7148,195.817 21.7148,196.275 21.7148,196.725 21.7148,197.169 21.7148,197.872 21.7148,198.946 
 21.7148,200.482 21.7148,201.93 21.7148,203.298 21.7148,204.588 21.7148,205.806 21.7148,206.955 21.7148,208.039 21.7148,209.769 21.7148,212.55 21.7148,214.861 
 21.7148,216.182 21.7148,217.349 21.7148,218.379 21.7148,219.734 21.7148,220.854 21.7148,221.78 21.7149,222.546 21.7149,223.179 21.7149,223.702 21.7149,224.134 
 21.7149,224.491 21.7149,224.787 21.7149,225.031 21.7149,225.233 21.7149,225.4 21.7149,225.538 21.7149,225.652 21.7149,225.791 21.7149,225.894 21.7149,225.971 
 21.7149,226.029 21.715,226.071 21.715,226.103 21.715,226.127 21.715,226.153 21.715,226.174 21.715,226.183 21.715,226.188 21.7151,226.191 21.7151,226.193 
 21.7151,226.194 21.7151,226.195 21.7151,226.195 21.7151,226.195 21.7151,226.196 21.7152,226.196 21.7152,226.196 21.7152,226.196 21.7152,226.196 21.7153,226.196 
 21.7153,226.196 21.7154,226.196 21.7154,226.196 21.7155,226.196 21.7156,226.196 21.7156,226.196 21.7158,226.196 21.716,226.196 21.7161,226.196 21.7165,226.196 
 21.7169,226.196 21.7172,226.196 21.7184,226.196 21.7205,226.197 21.7227,226.197 21.7442,226.201 21.8971,226.231 22.1294,226.276 22.3617,226.322 22.8363,226.415 
 23.311,226.51 23.7857,226.605 24.6954,226.79 26.2796,227.121 27.8637,227.463 29.4479,227.817 31.032,228.184 32.6162,228.564 34.2003,228.958 35.7845,229.368 
 37.7422,229.897 39.6999,230.452 41.6575,231.037 43.6152,231.654 45.5729,232.306 48.8177,233.473 50.8317,234.26 52.8457,235.1 54.8597,236.001 56.8737,236.97 
 58.8877,238.017 60.9017,239.153 62.9157,240.39 64.2019,241.241 65.488,242.147 66.7742,243.11 67.9331,244.035 69.0919,245.02 71.3551,247.142 72.4161,248.243 
 73.4771,249.421 74.538,250.686 75.599,252.052 76.66,253.53 77.7209,255.139 78.7819,256.898 79.8429,258.834 80.9038,260.977 81.4967,262.282 82.0896,263.672 
 82.6824,265.157 83.2753,266.749 83.8682,268.464 84.8662,271.671 85.3181,273.279 85.7701,274.998 86.222,276.844 86.6739,278.832 87.1259,280.982 87.5778,283.318 
 88.0298,285.867 88.4817,288.665 88.9337,291.756 89.1974,293.718 89.4611,295.81 89.7248,298.047 89.9885,300.45 90.2523,303.04 90.516,305.839 90.7797,308.882 
 91.0434,312.204 91.3071,315.852 91.5708,319.88 91.722,322.39 91.8732,325.066 92.102,329.471 92.2301,332.151 92.3583,335.006 92.4864,338.055 92.6145,341.322 
 92.7426,344.833 92.8708,348.622 92.9989,352.727 93.127,357.193 93.2551,362.079 93.3832,367.453 93.4583,370.864 93.5334,374.495 93.6084,378.365 93.6777,382.176 
 93.747,386.247 93.8561,393.252 93.9146,397.351 93.9731,401.726 94.0315,406.405 94.1036,412.64 94.1756,419.484 94.2476,427.045 94.3196,435.442 94.3698,441.878 
 94.42,448.881 94.4703,456.55 94.5205,464.965 94.5518,470.655 94.5832,476.736 94.6146,483.249 94.6459,490.245 94.6773,497.79 94.7086,505.961 94.74,514.843 
 94.7713,524.55 94.8027,535.222 94.834,547.025 94.8654,560.167 94.8968,574.93 94.9281,591.691 94.9595,610.929 94.9908,633.282 94.9941,635.88 94.9964,637.702 
 94.9987,639.528 95.001,641.375 95.0032,643.25 95.0055,645.151 95.0078,647.075 95.0101,649.023 95.0124,650.997 95.0146,652.998 95.0214,659.08 95.0281,665.413 
 95.0331,670.273 95.0381,675.29 95.0431,680.473 95.0481,685.827 95.0531,691.364 95.0581,697.098 95.0631,703.035 95.0712,713.136 95.0793,723.871 95.0874,735.297 
 95.0955,747.508 95.1036,760.622 95.1117,774.716 95.1175,785.479 95.1233,796.885 95.1276,805.705 95.1319,814.91 95.1362,824.542 95.1404,834.669 95.1447,845.291 
 95.1477,852.967 95.1506,860.9

In [9]:
sol = solve(prob,radau,abstol=abstols[3],reltol=reltols[3],maxiters=Int(1e5))
plot(sol,denseplot=false,ylim=[-4;4])

[DifferentialEquations.jl] Initializing backend: ODEInterface


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 21.7148,194.444 21.7148,195.383 21.7148,196.293 21.7148,200.006 21.7148,203.896 21.7148,208.288 21.7148,212.439 21.7148,215.628 21.7148,218.768 21.7148,220.975 
 21.7149,222.873 21.7149,224.08 21.7149,225.913 21.715,226.177 21.7151,226.195 21.7153,226.196 21.7159,226.196 21.7195,226.197 21.7486,226.202 21.8362,226.219 
 22.3434,226.318 24.6087,226.772 31.9371,228.399 45.6308,232.325 66.7946,243.123 80.2409,259.605 87.1637,281.147 90.9522,310.958 93.0031,352.696 94.1025,412.082 
 94.6845,498.292 94.9876,626.941 95.1201,780.609 95.189,975.937 95.2282,1238.69 95.2595,1861.2 95.2689,2385.14 95.275,3085.93 95.281,4706.02 95.2849,7661.62 
 95.2873,12409.8 95.2891,21100.1 95.2904,37581.6 95.2913,68272.9 95.292,125755 95.2925,234299 95.2929,440056 95.2931,832258 95.2933,1.58661e+06 95.2935,3.059e+06 
 95.2936,6.01275e+06 95.2937,1.23284e+07 95.2938,2.2518e+07 95.2938,4.23263e+07 95.2938,5.81055e+07 95.2939,6.01989e+07 95.2939,3.29258e+07 95.294,1.06649e+07 95.294,1.87565e+06 95.2941,485401 
 95.2941,81247.1 95.2942,16322.9 95.2942,3409.34 95.2943,819.524 95.2944,189.172 95.2945,163.762 95.2945,162.777 95.2947,162.695 95.295,162.695 95.2956,162.695 
 95.2979,162.694 95.3028,162.694 95.3188,162.69 95.3857,162.677 95.6091,162.634 96.1708,162.524 97.1209,162.335 99.6529,161.814 104.556,160.721 116.092,157.59 
 141.471,144.899 154.515,127.874 160.245,110.163 164.289,81.0188 166.547,37.7067 167.692,-23.1663 168.289,-113.162 168.591,-248.48 168.72,-412.906 168.786,-620.522 
 168.823,-914.623 168.852,-1582.48 168.861,-2186.45 168.866,-2981.52 168.871,-4772.61 168.875,-8077.87 168.877,-13509.7 168.879,-23461.8 168.88,-42296.6 168.881,-77450.5 
 168.882,-143393 168.882,-267998 168.882,-504457 168.883,-955916 168.883,-1.82645e+06 168.883,-3.53322e+06 168.883,-6.98869e+06 168.883,-1.4572e+07 168.883,-2.47882e+07 168.883,-4.09874e+07 
 168.883,-5.76465e+07 168.883,-5.97594e+07 168.883,-3.21445e+07 168.884,-1.07877e+07 168.884,-2.60934e+06 168.884,-808418 168.884,-152967 168.884,-36192.3 168.884,-8424.3 168.884,-128.813 
 168.884,211.626 168.884,225.596 168.884,226.169 168.884,226.194 168.885,226.194 168.886,226.194 168.891,226.195 168.91,226.199 169.025,226.221 169.414,226.297 
 170.318,226.475 171.795,226.774 174.945,227.441 180.052,228.627 189.969,231.389 209.912,240.271 225.097,255.448 233.575,277.572 237.869,307.762 240.112,350.409 
 241.273,411.312 241.874,500.971 242.179,636.097 242.309,799.943 242.375,1007.14 242.413,1298.9 242.441,1961.32 242.45,2560.55 242.456,3350.33 242.461,5126.73 
 242.465,8402.47 242.467,13782.4 242.468,23637.2 242.47,42287 242.471,77091.9 242.471,142374 242.472,265723 242.472,499779 242.472,946596 242.472,1.80801e+06 
 242.473,3.49631e+06 242.473,6.91188e+06 242.473,1.43913e+07 242.473,2.45664e+07 242.473,4.10472e+07 242.473,5.76928e+07 242.473,5.97474e+07 242.473,3.21363e+07 242.473,1.07935e+07 242.473,2.61157e+06 
 242.473,809574 242.473,153469 242.473,36628.8 242.473,8829.52 242.473,518.411 242.473,177.294 242.474,163.294 242.474,162.72 242.474,162.695 242.474,162.695 
 242.475,162.695 242.48,162.694 242.499,162.69 242.615,162.668 243.003,162.592 243.906,162.414 245.383,162.116 248.533,161.448 253.641,160.262 263.56,157.5 
 283.507,148.614 298.689,133.438 307.162,111.327 311.457,81.1485 313.7,38.5286 314.861,-22.3332 315.463,-111.92 315.768,-246.921 315.898,-410.59 315.964,-617.504 
 316.002,-908.761 316.03,-1570.78 316.039,-2169.15 316.045,-2957.24 316.05,-4730.63 316.054,-8001.25 316.056,-13371 316.058,-23206.4 316.059,-41820.1 316.06,-76557 
 316.061,-141709 316.061,-264813 316.062,-498400 316.062,-944309 316.062,-1.80394e+06 316.062,-3.48863e+06 316.062,-6.89642e+06 316.062,-1.43556e+07 316.062,-2.45227e+07 316

In [2]:
setups = [Dict(:alg=>Rosenbrock23),
          Dict(:alg=>ModifiedRosenbrockIntegrator),
          Dict(:alg=>CVODE_BDF),
          Dict(:alg=>radau)]
wp = ode_workprecision_set(prob,abstols,reltols,setups;dense=false,save_timeseries=false,appxsol=test_sol,maxiters=Int(1e5))
plot(wp)

[DifferentialEquations.jl] Initializing backend: ODEJL
[DifferentialEquations.jl] Initializing backend: ODEInterface


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 6 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 DiffEqBase.Rosenbrock23 
 
 
 
 DiffEqBase.ModifiedRosenbrockIntegrator 
 
 
 
 DiffEqBase.CVODE 
 
 
 BDF 
 
 
 
 DiffEqBase.radau

### Conclusion

Of the Julia-based methods, Rosenbrock23 is the better method. It is both faster and has less error. CVODE_BDF is faster, while radau solves with much lower error.